In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.Trainer.Trainer import Trainer
from src.Models.Model_class import Model_class

from src.Trainer.Trainer import Trainer
from src.Losses.Focal_loss import FocalLoss
from src.Models.Model_class import Model_class
from src.Losses.Loss_class import Loss_class

from src.Models.Autoenc_simpl import Autoencoder
from src.Models.Classifier import Simple_classifier
from src.Models.JoinedModel import JoinedModel

from src.DatasetsClasses.EncDataset import EncoderDataset
from src.Losses.Loss_class_encoder import Loss_class_encoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df = pd.read_csv('../data/df_to_enc.csv')
df = df.drop(columns = ['Machine failure'])

In [3]:
HIDDEN_PARAM = 128
LATENT_REPR = 20

BATCH_SIZE = 1024

In [4]:
X_train, X_test = train_test_split(df, shuffle=True, random_state=42)

In [5]:
train_dataset = EncoderDataset(X_train)
val_dataset = EncoderDataset(X_test)

In [6]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [7]:
autoencoder = Autoencoder(df.shape[1], HIDDEN_PARAM, LATENT_REPR)

In [8]:
loss = Loss_class_encoder(nn.MSELoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=autoencoder,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=10)

wandb_init_params = dict(
    name=f'Autoencoder_simple_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [9]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/',
                  saving_model=True
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [10]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:   0%|          | 0/8 [00:00<?, ?it/s]

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 81.80it/s]


Epoch: 1 of 10, 0.012 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 70.25it/s]


Epoch: 2 of 10, 0.007 min


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 74.37it/s]


Epoch: 9 of 10, 0.010 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 64.13it/s]


Epoch: 10 of 10, 0.008 min


epoch,▁▂▃▃▄▅▆▆▇█
lr,█▇▆▅▄▄▃▂▂▁
test_loss,█▄▃▂▁▁▁▁▁▁
train_loss,█▃▂▂▁▁▁▁▁▁
epoch,9
lr,0.00599
test_loss,0.0617
train_loss,0.06179
